#Tool-calling Agent

This is an auto-generated notebook created by an AI playground export. In this notebook, you will:
- Author a tool-calling [MLflow's `ResponsesAgent`](https://mlflow.org/docs/latest/api_reference/python_api/mlflow.pyfunc.html#mlflow.pyfunc.ResponsesAgent) that uses the OpenAI client
- Manually test the agent's output
- Evaluate the agent with Mosaic AI Agent Evaluation
- Log and deploy the agent

This notebook should be run on the `GenAI Agents Cluster` (preferred) or another cluster with DBR < 17. If serverless is enabled, select a cluster before running.

 **_NOTE:_**  This notebook uses the OpenAI SDK, but AI Agent Framework is compatible with any agent authoring framework, including LlamaIndex or LangGraph. To learn more, see the [Authoring Agents](https://learn.microsoft.com/azure/databricks/generative-ai/agent-framework/author-agent) Databricks documentation.

## Prerequisites

- Address all `TODO`s in this notebook.

## Define the agent in code
Below we define our agent code in a single cell, enabling us to easily write it to a local Python file for subsequent logging and deployment using the `%%writefile` magic command.

For more examples of tools to add to your agent, see [docs](https://learn.microsoft.com/azure/databricks/generative-ai/agent-framework/agent-tool).

In [0]:
%%writefile agent.py
import json
from typing import Any, Callable, Generator, Optional
from uuid import uuid4
import warnings

import mlflow
from databricks.sdk import WorkspaceClient
from databricks_openai import UCFunctionToolkit
from mlflow.entities import SpanType
from mlflow.pyfunc import ResponsesAgent
from mlflow.types.responses import (
    ResponsesAgentRequest,
    ResponsesAgentResponse,
    ResponsesAgentStreamEvent,
    output_to_responses_items_stream,
    to_chat_completions_input,
)
from openai import OpenAI
from pydantic import BaseModel
from unitycatalog.ai.core.base import get_uc_function_client

############################################
# Define your LLM endpoint and system prompt
############################################
LLM_ENDPOINT_NAME = "databricks-gpt-oss-120b"

SYSTEM_PROMPT = (
    "You are a helpful Databricks assistant. "
    "Use tools when they add value, and respond concisely. "
    "If a tool call fails or data is missing, say so."
)


###############################################################################
## Define tools for your agent, enabling it to retrieve data or take actions
## beyond text generation
## To create and see usage examples of more tools, see
## https://learn.microsoft.com/azure/databricks/generative-ai/agent-framework/agent-tool
###############################################################################
class ToolInfo(BaseModel):
    """
    Class representing a tool for the agent.
    - "name" (str): The name of the tool.
    - "spec" (dict): JSON description of the tool (matches OpenAI Responses format)
    - "exec_fn" (Callable): Function that implements the tool logic
    """

    name: str
    spec: dict
    exec_fn: Callable


def create_tool_info(tool_spec, exec_fn_param: Optional[Callable] = None):
    tool_spec["function"].pop("strict", None)
    tool_name = tool_spec["function"]["name"]
    udf_name = tool_name.replace("__", ".")

    # Define a wrapper that accepts kwargs for the UC tool call,
    # then passes them to the UC tool execution client
    def exec_fn(**kwargs):
        function_result = uc_function_client.execute_function(udf_name, kwargs)
        if function_result.error is not None:
            return function_result.error
        else:
            return function_result.value
    return ToolInfo(name=tool_name, spec=tool_spec, exec_fn=exec_fn_param or exec_fn)


TOOL_INFOS = []

# You can use UDFs in Unity Catalog as agent tools
# TODO: Add additional tools
UC_TOOL_NAMES = ["system.ai.python_exec"]

uc_toolkit = UCFunctionToolkit(function_names=UC_TOOL_NAMES)
uc_function_client = get_uc_function_client()
for tool_spec in uc_toolkit.tools:
    TOOL_INFOS.append(create_tool_info(tool_spec))


# Custom tools defined in this notebook
def get_utc_timestamp(**_):
    from datetime import datetime, timezone
    return datetime.now(timezone.utc).isoformat()


def word_count(text: str, **_):
    return len(text.split())


def echo_text(text: str, **_):
    return text


CUSTOM_TOOLS = [
    (
        {
            "type": "function",
            "function": {
                "name": "get_utc_timestamp",
                "description": "Return the current UTC timestamp in ISO 8601 format.",
                "parameters": {"type": "object", "properties": {}, "required": []},
            },
        },
        get_utc_timestamp,
    ),
    (
        {
            "type": "function",
            "function": {
                "name": "word_count",
                "description": "Count the number of words in the provided text.",
                "parameters": {
                    "type": "object",
                    "properties": {"text": {"type": "string"}},
                    "required": ["text"],
                },
            },
        },
        word_count,
    ),
    (
        {
            "type": "function",
            "function": {
                "name": "echo_text",
                "description": "Echo the provided text.",
                "parameters": {
                    "type": "object",
                    "properties": {"text": {"type": "string"}},
                    "required": ["text"],
                },
            },
        },
        echo_text,
    ),
]


for tool_spec, exec_fn in CUSTOM_TOOLS:
    TOOL_INFOS.append(create_tool_info(tool_spec, exec_fn_param=exec_fn))




class ToolCallingAgent(ResponsesAgent):
    """
    Class representing a tool-calling Agent
    """

    def __init__(self, llm_endpoint: str, tools: list[ToolInfo]):
        """Initializes the ToolCallingAgent with tools."""
        self.llm_endpoint = llm_endpoint
        self.workspace_client = WorkspaceClient()
        self.model_serving_client: OpenAI = (
            self.workspace_client.serving_endpoints.get_open_ai_client()
        )
        self._tools_dict = {tool.name: tool for tool in tools}

    def get_tool_specs(self) -> list[dict]:
        """Returns tool specifications in the format OpenAI expects."""
        return [tool_info.spec for tool_info in self._tools_dict.values()]

    @mlflow.trace(span_type=SpanType.TOOL)
    def execute_tool(self, tool_name: str, args: dict) -> Any:
        """Executes the specified tool with the given arguments."""
        return self._tools_dict[tool_name].exec_fn(**args)

    def call_llm(self, messages: list[dict[str, Any]]) -> Generator[dict[str, Any], None, None]:
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore", message="PydanticSerializationUnexpectedValue")
            for chunk in self.model_serving_client.chat.completions.create(
                model=self.llm_endpoint,
                messages=to_chat_completions_input(messages),
                tools=self.get_tool_specs(),
                stream=True,
            ):
                chunk_dict = chunk.to_dict()
                if len(chunk_dict.get("choices", [])) > 0:
                    yield chunk_dict

    def handle_tool_call(
        self,
        tool_call: dict[str, Any],
        messages: list[dict[str, Any]],
    ) -> ResponsesAgentStreamEvent:
        """
        Execute tool calls, add them to the running message history, and return a ResponsesStreamEvent w/ tool output
        """
        args = json.loads(tool_call["arguments"])
        result = str(self.execute_tool(tool_name=tool_call["name"], args=args))

        tool_call_output = self.create_function_call_output_item(tool_call["call_id"], result)
        messages.append(tool_call_output)
        return ResponsesAgentStreamEvent(type="response.output_item.done", item=tool_call_output)

    def call_and_run_tools(
        self,
        messages: list[dict[str, Any]],
        max_iter: int = 10,
    ) -> Generator[ResponsesAgentStreamEvent, None, None]:
        for _ in range(max_iter):
            last_msg = messages[-1]
            if last_msg.get("role", None) == "assistant":
                return
            elif last_msg.get("type", None) == "function_call":
                yield self.handle_tool_call(last_msg, messages)
            else:
                yield from output_to_responses_items_stream(
                    chunks=self.call_llm(messages), aggregator=messages
                )

        yield ResponsesAgentStreamEvent(
            type="response.output_item.done",
            item=self.create_text_output_item("Max iterations reached. Stopping.", str(uuid4())),
        )

    def predict(self, request: ResponsesAgentRequest) -> ResponsesAgentResponse:
        outputs = [
            event.item
            for event in self.predict_stream(request)
            if event.type == "response.output_item.done"
        ]
        return ResponsesAgentResponse(output=outputs, custom_outputs=request.custom_inputs)

    def predict_stream(
        self, request: ResponsesAgentRequest
    ) -> Generator[ResponsesAgentStreamEvent, None, None]:
        messages = to_chat_completions_input([i.model_dump() for i in request.input])
        if SYSTEM_PROMPT:
            messages.insert(0, {"role": "system", "content": SYSTEM_PROMPT})
        yield from self.call_and_run_tools(messages=messages)


# Log the model using MLflow
mlflow.openai.autolog()
AGENT = ToolCallingAgent(llm_endpoint=LLM_ENDPOINT_NAME, tools=TOOL_INFOS)
mlflow.models.set_model(AGENT)


Overwriting agent.py


## Test the agent

Interact with the agent to test its output. Since we manually traced methods within `ResponsesAgent`, you can view the trace for each step the agent takes, with any LLM calls made via the OpenAI SDK automatically traced by autologging.

Replace this placeholder input with tool-focused examples that force tool usage so you can inspect traces.


In [0]:
import warnings
from agent import AGENT

warnings.filterwarnings("ignore", message="PydanticSerializationUnexpectedValue")

tests = [
    {"role": "user", "content": "Use get_utc_timestamp to return the current UTC timestamp in ISO-8601."},
    {"role": "user", "content": "Use word_count to count the words in: 'Databricks agents are fun to test'."},
    {"role": "user", "content": "Use echo_text to repeat exactly: tools are working."},
    {"role": "user", "content": "Using python_exec, compute 19*23 and return just the number."},
]

def summarize_response(response):
    for item in response.output:
        if item.type == "function_call":
            print(f"tool_call: {item.name} args={item.arguments}")
        elif item.type == "function_call_output":
            print(f"tool_output: {item.output}")
        elif item.type == "message":
            text = ""
            for chunk in item.content or []:
                if isinstance(chunk, dict) and chunk.get("type") == "output_text":
                    text = chunk.get("text", "")
                    break
            print(f"assistant: {text}")

for t in tests:
    resp = AGENT.predict({"input": [t]})
    summarize_response(resp)
    print("---")


/local_disk0/.ephemeral_nfs/cluster_libraries/python/lib/python3.12/site-packages/pydantic/main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `str` - serialized value may not be as expected [field_name='content', input_value=[{'type': 'reasoning', 's...15:19:35.573538+00:00'}], input_type=list])
  return self.__pydantic_serializer__.to_python(


tool_call: get_utc_timestamp args={}
tool_output: 2026-01-02T15:19:35.573538+00:00
assistant: 2026-01-02T15:19:35.573538+00:00
---
tool_call: word_count args={
  "text": "Databricks agents are fun to test"
}
tool_output: 6
assistant: The text contains **6 words**.
---


/local_disk0/.ephemeral_nfs/cluster_libraries/python/lib/python3.12/site-packages/pydantic/main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `str` - serialized value may not be as expected [field_name='content', input_value=[{'type': 'reasoning', 's...*tools are working.**'}], input_type=list])
  return self.__pydantic_serializer__.to_python(


tool_call: echo_text args={
  "text": "tools are working."
}
tool_output: tools are working.
assistant: Here you go:

**tools are working.**
---
tool_call: system__ai__python_exec args={
  "code": "print(19*23)"
}
tool_output: 437

assistant: 437
---


[Trace(trace_id=tr-e3aa5ad6be99fc0d360d192882f8ab62), Trace(trace_id=tr-1886efabd58ea41b53fea7ae97f86429), Trace(trace_id=tr-7cefd471ac113c55aaea8feb37ea3e2b), Trace(trace_id=tr-94e7b71ef3de82a4a8c98fb5dc05ccdf)]

In [0]:
for chunk in AGENT.predict_stream(
    {"input": [{"role": "user", "content": "Using python_exec, compute 19*23 and return just the number."}]}
):
    print(chunk.model_dump(exclude_none=True))


{'type': 'response.output_item.done', 'item': {'type': 'function_call', 'id': 'chatcmpl_f696b2e9-2ccd-41da-94ec-073585b952d3', 'call_id': 'call_81bfb75e-1c92-495b-9a63-a6286e867e8e', 'name': 'system__ai__python_exec', 'arguments': '{\n  "code": "print(19*23)"\n}'}}
{'type': 'response.output_item.done', 'item': {'type': 'function_call_output', 'call_id': 'call_81bfb75e-1c92-495b-9a63-a6286e867e8e', 'output': '437\n'}}
{'type': 'response.output_text.delta', 'item_id': 'chatcmpl_9cd25eb4-37e2-4306-aa6b-f1ab55c96eca', 'delta': ''}
{'type': 'response.output_text.delta', 'item_id': 'chatcmpl_9cd25eb4-37e2-4306-aa6b-f1ab55c96eca', 'delta': '437'}
{'type': 'response.output_text.delta', 'item_id': 'chatcmpl_9cd25eb4-37e2-4306-aa6b-f1ab55c96eca', 'delta': ''}
{'type': 'response.output_item.done', 'item': {'id': 'chatcmpl_9cd25eb4-37e2-4306-aa6b-f1ab55c96eca', 'content': [{'text': '437', 'type': 'output_text'}], 'role': 'assistant', 'type': 'message'}}


Trace(trace_id=tr-17577afcc7d58f0d7d02b70cd876df3d)

### Log the `agent` as an MLflow model
Determine Databricks resources to specify for automatic auth passthrough at deployment time
- **TODO**: If your Unity Catalog Function queries a [vector search index](https://learn.microsoft.com/azure/databricks/generative-ai/agent-framework/unstructured-retrieval-tools) or leverages [external functions](https://learn.microsoft.com/azure/databricks/generative-ai/agent-framework/external-connection-tools), you need to include the dependent vector search index and UC connection objects, respectively, as resources. See [docs](https://learn.microsoft.com/azure/databricks/generative-ai/agent-framework/log-agent#specify-resources-for-automatic-authentication-passthrough) for more details.

Log the agent as code from the `agent.py` file. See [MLflow - Models from Code](https://mlflow.org/docs/latest/models.html#models-from-code).

In [0]:
# Determine Databricks resources to specify for automatic auth passthrough at deployment time
import logging
import warnings

import mlflow
from agent import UC_TOOL_NAMES, LLM_ENDPOINT_NAME
from mlflow.models.resources import DatabricksFunction, DatabricksServingEndpoint
from pkg_resources import get_distribution

logging.getLogger("mlflow.tracing.fluent").setLevel(logging.ERROR)
warnings.filterwarnings("ignore", message="PydanticSerializationUnexpectedValue")

resources = [DatabricksServingEndpoint(endpoint_name=LLM_ENDPOINT_NAME)]
for tool_name in UC_TOOL_NAMES:
    # TODO: If the UC function includes dependencies like external connection or vector search, please include them manually.
    # See the TODO in the markdown above for more information.
    resources.append(DatabricksFunction(function_name=tool_name))

input_example = {
    "input": [
        {
            "role": "user",
            "content": "What is an LLM agent?"
        }
    ]
}

with mlflow.start_run():
    logged_agent_info = mlflow.pyfunc.log_model(
        name="agent",
        python_model="agent.py",
        input_example=input_example,
        pip_requirements=[
            "databricks-openai",
            "backoff",
            f"databricks-connect=={get_distribution('databricks-connect').version}",
        ],
        resources=resources,
    )


🔗 View Logged Model at: https://adb-7405615340160596.16.azuredatabricks.net/ml/experiments/463397721036728/models/m-7cb2a565730e4d0cbcda83908c4ac991?o=7405615340160596
2026/01/02 15:20:05 INFO mlflow.pyfunc: Predicting on input example to validate output
/local_disk0/.ephemeral_nfs/cluster_libraries/python/lib/python3.12/site-packages/pydantic/main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `str` - serialized value may not be as expected [field_name='content', input_value=[{'type': 'reasoning', 's...mple text generation.'}], input_type=list])
  return self.__pydantic_serializer__.to_python(
/local_disk0/.ephemeral_nfs/cluster_libraries/python/lib/python3.12/site-packages/pydantic/main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `str` - serialized value may not be as expected [field_name='content', input_value=[{'type': 'reasoning', 's...nerating static text.'}], input_type=lis

## Evaluate the agent with [Agent Evaluation](https://learn.microsoft.com/azure/databricks/mlflow3/genai/eval-monitor)

You can edit the requests or expected responses in your evaluation dataset and run evaluation as you iterate your agent, leveraging mlflow to track the computed quality metrics.

Evaluate your agent with one of our [predefined LLM scorers](https://learn.microsoft.com/azure/databricks/mlflow3/genai/eval-monitor/predefined-judge-scorers), or try adding [custom metrics](https://learn.microsoft.com/azure/databricks/mlflow3/genai/eval-monitor/custom-scorers).

In [0]:
import mlflow
from mlflow.genai.scorers import RelevanceToQuery, Safety, RetrievalRelevance, RetrievalGroundedness

eval_dataset = [
    {
        "inputs": {
            "input": [
                {
                    "role": "user",
                    "content": "What is an LLM agent?"
                }
            ]
        },
        "expected_response": None
    }
]

eval_results = mlflow.genai.evaluate(
    data=eval_dataset,
    predict_fn=lambda input: AGENT.predict({"input": input}),
    scorers=[RelevanceToQuery(), Safety()], # add more scorers here if they're applicable
)

# Review the evaluation results in the MLfLow UI (see console output)

2026/01/02 15:20:15 INFO mlflow.genai.utils.data_validation: Testing model prediction with the first sample in the dataset. To disable this check, set the MLFLOW_GENAI_EVAL_SKIP_TRACE_VALIDATION environment variable to True.
/local_disk0/.ephemeral_nfs/cluster_libraries/python/lib/python3.12/site-packages/pydantic/main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `str` - serialized value may not be as expected [field_name='content', input_value=[{'type': 'reasoning', 's...orms like Databricks.'}], input_type=list])
  return self.__pydantic_serializer__.to_python(


Evaluating:   0%|          | 0/1 [Elapsed: 00:00, Remaining: ?] 

/local_disk0/.ephemeral_nfs/cluster_libraries/python/lib/python3.12/site-packages/pydantic/main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `str` - serialized value may not be as expected [field_name='content', input_value=[{'type': 'reasoning', 's...generate static text.'}], input_type=list])
  return self.__pydantic_serializer__.to_python(


## Perform pre-deployment validation of the agent
Before registering and deploying the agent, we perform pre-deployment checks via the [mlflow.models.predict()](https://mlflow.org/docs/latest/python_api/mlflow.models.html#mlflow.models.predict) API. See [documentation](https://learn.microsoft.com/azure/databricks/machine-learning/model-serving/model-serving-debug#validate-inputs) for details

In [0]:
mlflow.models.predict(
    model_uri=f"runs:/{logged_agent_info.run_id}/agent",
    input_data={"input": [{"role": "user", "content": "Hello!"}]},
    env_manager="uv",
)

2026/01/02 15:20:32 INFO mlflow.models.flavor_backend_registry: Selected backend for flavor 'python_function'


2026/01/02 15:20:34 INFO mlflow.utils.virtualenv: Environment /local_disk0/.ephemeral_nfs/repl_tmp_data/ReplId-19b7f-1bc68-5/mlflow/envs/virtualenv_envs/mlflow-e9d6f2e194b4f6fdbaf6eeed2aefd5e24332888f already exists
2026/01/02 15:20:34 INFO mlflow.utils.environment: === Running command '['bash', '-c', 'source /local_disk0/.ephemeral_nfs/repl_tmp_data/ReplId-19b7f-1bc68-5/mlflow/envs/virtualenv_envs/mlflow-e9d6f2e194b4f6fdbaf6eeed2aefd5e24332888f/bin/activate && python -c ""']'
2026/01/02 15:20:34 INFO mlflow.utils.environment: === Running command '['bash', '-c', 'source /local_disk0/.ephemeral_nfs/repl_tmp_data/ReplId-19b7f-1bc68-5/mlflow/envs/virtualenv_envs/mlflow-e9d6f2e194b4f6fdbaf6eeed2aefd5e24332888f/bin/activate && python /local_disk0/.ephemeral_nfs/cluster_libraries/python/lib/python3.12/site-packages/mlflow/pyfunc/_mlflow_pyfunc_backend_predict.py --model-uri file:///local_disk0/repl_tmp_data/ReplId-19b7f-1bc68-5/tmpukiy3ka2/agent --content-type json --input-path /local_disk0/

{"object": "response", "output": [{"type": "reasoning", "summary": [{"type": "summary_text", "text": "The user just says \"Hello!\". As a Databricks assistant, we respond politely. No tool needed."}], "id": "chatcmpl_3879e3ff-dae2-4d11-aef8-0d34e6228c13"}, {"type": "message", "id": "chatcmpl_3879e3ff-dae2-4d11-aef8-0d34e6228c13", "content": [{"text": "Hello! How can I help you today?", "type": "output_text"}], "role": "assistant"}]}

2026/01/02 15:20:49 INFO mlflow.tracing.export.async_export_queue: Flushing the async trace logging queue before program exit. This may take a while...


## Register the model to Unity Catalog

Update the `catalog`, `schema`, and `model_name` below to register the MLflow model to Unity Catalog.

In [0]:
mlflow.set_registry_uri("databricks-uc")

# Auto-select a usable catalog/schema for UC registration
catalogs = [r.catalog for r in spark.sql("SHOW CATALOGS").collect()]
current_catalog = None
try:
    current_catalog = spark.sql("SELECT current_catalog()").first()[0]
except Exception:
    current_catalog = None

candidates = [c for c in catalogs if c not in ("system", "samples")]
if current_catalog in candidates:
    candidates.remove(current_catalog)
    candidates.insert(0, current_catalog)

schema = "default"
catalog = None
last_error = None

def can_use_catalog(candidate):
    try:
        spark.sql(f"USE CATALOG `{candidate}`")
        spark.sql("SHOW SCHEMAS").collect()
        return True
    except Exception as exc:
        return False

for candidate in candidates:
    if not can_use_catalog(candidate):
        continue
    try:
        spark.sql(f"CREATE SCHEMA IF NOT EXISTS `{candidate}`.`{schema}`")
        catalog = candidate
        break
    except Exception as exc:
        last_error = exc

if catalog is None:
    raise RuntimeError(
        "No usable catalog found for UC registration. "
        "Ask an admin to grant USE CATALOG/CREATE SCHEMA, "
        f"or check permissions. Last error: {last_error}"
    )

model_name = "genai-agent"
UC_MODEL_NAME = f"{catalog}.{schema}.{model_name}"
print(f"Registering model to {UC_MODEL_NAME}")

# register the model to UC
uc_registered_model_info = mlflow.register_model(
    model_uri=logged_agent_info.model_uri, name=UC_MODEL_NAME
)


Registering model to adb_genai_evolved_teal.default.genai-agent


Successfully registered model 'adb_genai_evolved_teal.default.genai-agent'.


Uploading artifacts:   0%|          | 0/13 [00:00<?, ?it/s]

🔗 Created version '1' of model 'adb_genai_evolved_teal.default.genai-agent': https://adb-7405615340160596.16.azuredatabricks.net/explore/data/models/adb_genai_evolved_teal/default/genai-agent/version/1?o=7405615340160596


## Deploy the agent

In [0]:
from databricks import agents
# NOTE: pass scale_to_zero=True to agents.deploy() to enable scale-to-zero for cost savings.
# This is not recommended for production workloads, as capacity is not guaranteed when scaled to zero.
# Scaled to zero endpoints may take extra time to respond when queried, while they scale back up.
agents.deploy(UC_MODEL_NAME, uc_registered_model_info.version, tags = {"endpointSource": "playground"})

/local_disk0/.ephemeral_nfs/cluster_libraries/python/lib/python3.12/site-packages/databricks/agents/deployments.py:641: UserWarning: This endpoint is being deployed without a feedback model, which has been deprecated.
For more information, see: https://docs.databricks.com/aws/en/generative-ai/agent-framework/feedback-model
  warnings.warn(



    Deployment of adb_genai_evolved_teal.default.genai-agent version 1 initiated.  This can take up to 15 minutes and the Review App & Query Endpoint will not work until this deployment finishes.

    View status: https://adb-7405615340160596.16.azuredatabricks.net/ml/endpoints/agents_adb_genai_evolved_teal-default-genai-agent/?o=7405615340160596
    Review App: https://adb-7405615340160596.16.azuredatabricks.net/ml/review-v2/439fae7ccbef4a6da8765701466c850d/chat?o=7405615340160596

You can refer back to the links above from the endpoint detail page at https://adb-7405615340160596.16.azuredatabricks.net/ml/endpoints/agents_adb_genai_evolved_teal-default-genai-agent/?o=7405615340160596.

To set up monitoring for your deployed agent, see:
https://docs.databricks.com/aws/en/mlflow3/genai/eval-monitor/production-monitoring


Deployment(model_name='adb_genai_evolved_teal.default.genai-agent', model_version='1', endpoint_name='agents_adb_genai_evolved_teal-default-genai-agent', served_entity_name='adb_genai_evolved_teal-default-genai-agent_1', query_endpoint='https://adb-7405615340160596.16.azuredatabricks.net/serving-endpoints/agents_adb_genai_evolved_teal-default-genai-agent/served-models/adb_genai_evolved_teal-default-genai-agent_1/invocations?o=7405615340160596', endpoint_url='https://adb-7405615340160596.16.azuredatabricks.net/ml/endpoints/agents_adb_genai_evolved_teal-default-genai-agent/?o=7405615340160596', review_app_url='https://adb-7405615340160596.16.azuredatabricks.net/ml/review-v2/439fae7ccbef4a6da8765701466c850d/chat?o=7405615340160596')

## Next steps

After your agent is deployed, you can chat with it in AI playground to perform additional checks, share it with SMEs in your organization for feedback, or embed it in a production application. See [docs](https://learn.microsoft.com/azure/databricks/generative-ai/deploy-agent) for details